In [1]:
!pip install openai==0.28.1
!pip install fpdf==1.7.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=cbe2e22c7f02ab4c15c444d0fb207393377a081e036b82e79c4736d932c78dda
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [2]:
import pandas as pd
import openai
import time
from fpdf import FPDF

openai.api_key = '' #ENTER API TOKEN HERE

## Load the dataset

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/XCai777/AI_Republic_Bootcamp/refs/heads/main/TruckKun/truckkun.csv')
df.head()

,Parcel ID,Sender Name,Sender Address,Recipient Name,Recipient Address,Weight,Dimensions,Shipping Method,Parcel Type,Insurance,Signature Required,Delivery Status,Expected Delivery Date,Tracking Number,Shipment Date,Delivery Date,Delivery Fees,Payment Status,Courier Name
0,P605730,John Doe,"123 Sender St, City 5",Jane Smith,"456 Recipient Rd, City 10",5kg - 10kg,Small (30cm x 30cm),Express,Document,Yes,Yes,Out for Delivery,2024-11-15,TRK10001,2024-11-12,2024-11-14,$15.00,Paid,Mike Johnson
1,P812503,Sarah Lee,"98 Sender Ave, City 3",Bob Marley,"765 Recipient Blvd, City 6",Over 10kg,Medium (50cm x 50cm),Standard,Box,Yes,No,Cancelled,2024-11-16,TRK10002,2024-11-09,2024-11-16,$25.00,Paid,Lucy Walker
2,P350514,Mary Green,"231 Sender Rd, City 8",Tom Hardy,"1032 Recipient Ln, City 4",1kg - 5kg,Large (100cm x 100cm),Overnight,Envelope,No,Yes,Delivered,2024-11-18,TRK10003,2024-11-11,2024-11-18,$30.00,Pending,Jack Hughes
3,P821742,Alex Turner,"890 Sender St, City 4",Emily Davis,"228 Recipient Blvd, City 7",5kg - 10kg,Medium (50cm x 50cm),Express,Pallet,Yes,No,Out for Delivery,2024-11-14,TRK10004,2024-11-13,2024-11-14,$40.00,Paid,Fiona White
4,P493428,Emily Clark,"563 Sender Blvd, City 9",Chris Martin,"278 Recipient Dr, City 2",Up to 1kg,Small (30cm x 30cm),Standard,Document,Yes,Yes,Pending,2024-11-20,TRK10005,2024-11-12,NaN,$10.00,Pending,Mark Thompson


In [44]:
df['Delivery Fees'] = df['Delivery Fees'].str.replace('$', '').astype(float)

In [45]:
df.sample(5)

,Parcel ID,Sender Name,Sender Address,Recipient Name,Recipient Address,Weight,Dimensions,Shipping Method,Parcel Type,Insurance,Signature Required,Delivery Status,Expected Delivery Date,Tracking Number,Shipment Date,Delivery Date,Delivery Fees,Payment Status,Courier Name,paragraph
1,P812503,Sarah Lee,"98 Sender Ave, City 3",Bob Marley,"765 Recipient Blvd, City 6",Over 10kg,Medium (50cm x 50cm),Standard,Box,Yes,No,Cancelled,2024-11-16,TRK10002,2024-11-09,2024-11-16,25.0,Paid,Lucy Walker,"""{column_summary} has a total of {unique_value..."
8,P706314,Jessica Fox,"800 Sender Ln, City 12",Paul Walker,"782 Recipient Blvd, City 10",5kg - 10kg,Large (100cm x 100cm),Express,Box,Yes,No,Out for Delivery,2024-11-17,TRK10009,2024-11-13,2024-11-16,35.0,Pending,Daniel White,"""{column_summary} has a total of {unique_value..."
7,P507362,Robert King,"231 Sender Blvd, City 1",Rachel Green,"905 Recipient St, City 9",1kg - 5kg,Small (30cm x 30cm),Standard,Envelope,No,Yes,Delivered,2024-11-19,TRK10008,2024-11-12,2024-11-18,12.0,Pending,Carl Foster,"""{column_summary} has a total of {unique_value..."
17,P902536,Natalie Rose,"294 Sender Rd, City 13",Tony Stark,"788 Recipient Ln, City 17",Up to 1kg,Medium (50cm x 50cm),Standard,Box,Yes,No,In Transit,2024-11-21,TRK10018,2024-11-10,2024-11-18,18.0,Paid,Brian White,"""{column_summary} has a total of {unique_value..."
20,P988511,Von Ayads,123 Something St. Something City,Mike Bambroz,423 Somewhere St. Something City,1kg - 5kg,Small (30cm x 30cm),Standard,Document,Yes,Yes,Pending,2024-11-17,TRK66538,2024-11-15,NaN,15.0,Unpaid,Mike Johnson,"""{column_summary} has a total of {unique_value..."


## Understanding the Dataset

In [46]:
# Get Column Names and Data Types
column_info = df.dtypes
print(column_info)

Parcel ID                  object
Sender Name                object
Sender Address             object
Recipient Name             object
Recipient Address          object
Weight                     object
Dimensions                 object
Shipping Method            object
Parcel Type                object
Insurance                  object
Signature Required         object
Delivery Status            object
Expected Delivery Date     object
Tracking Number            object
Shipment Date              object
Delivery Date              object
Delivery Fees             float64
Payment Status             object
Courier Name               object
paragraph                  object
dtype: object


In [47]:
# Identify Categorical and Numerical Features
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns

print("Categorical Features:", categorical_features)
print("Numerical Features:", numerical_features)


Categorical Features: Index(['Parcel ID', 'Sender Name', 'Sender Address', 'Recipient Name',
       'Recipient Address', 'Weight', 'Dimensions', 'Shipping Method',
       'Parcel Type', 'Insurance', 'Signature Required', 'Delivery Status',
       'Expected Delivery Date', 'Tracking Number', 'Shipment Date',
       'Delivery Date', 'Payment Status', 'Courier Name', 'paragraph'],
      dtype='object')
Numerical Features: Index(['Delivery Fees'], dtype='object')


In [48]:
# Get Unique Values for Categorical Features with Limitation
def get_unique_values(col, max_display=10):
    uniques = df[col].unique().tolist()
    if len(uniques) > max_display:
        return uniques[:max_display] + ["..."]
    return uniques

unique_values = {col: get_unique_values(col) for col in categorical_features}

print("Unique Values for Categorical Features:")
for col, uniques in unique_values.items():
    print(f" - {col}: {uniques}")
print()



Unique Values for Categorical Features:
 - Parcel ID: ['P605730', 'P812503', 'P350514', 'P821742', 'P493428', 'P776725', 'P766420', 'P507362', 'P706314', 'P305969', '...']
 - Sender Name: ['John Doe', 'Sarah Lee', 'Mary Green', 'Alex Turner', 'Emily Clark', 'Jason Brown', 'Laura Wilson', 'Robert King', 'Jessica Fox', 'Kevin White', '...']
 - Sender Address: ['123 Sender St, City 5', '98 Sender Ave, City 3', '231 Sender Rd, City 8', '890 Sender St, City 4', '563 Sender Blvd, City 9', '120 Sender Rd, City 6', '407 Sender Rd, City 2', '231 Sender Blvd, City 1', '800 Sender Ln, City 12', '431 Sender Rd, City 3', '...']
 - Recipient Name: ['Jane Smith', 'Bob Marley', 'Tom Hardy', 'Emily Davis', 'Chris Martin', 'Diana Rose', 'David Smith', 'Rachel Green', 'Paul Walker', 'Laura King', '...']
 - Recipient Address: ['456 Recipient Rd, City 10', '765 Recipient Blvd, City 6', '1032 Recipient Ln, City 4', '228 Recipient Blvd, City 7', '278 Recipient Dr, City 2', '389 Recipient Ave, City 3', '578 R

In [49]:
# Get Descriptive Statistics for Numerical Features
numerical_stats = df[numerical_features].describe()
print("Descriptive Statistics for Numerical Features:\n", numerical_stats, "\n")


Descriptive Statistics for Numerical Features:
        Delivery Fees
count      24.000000
mean       26.041667
std        14.195771
min        10.000000
25%        15.000000
50%        20.000000
75%        31.250000
max        60.000000 



## Generating the Template

In [56]:
def generate_template(df, column_info, categorical_features, numerical_features, numerical_stats, unique_values):

    # Construct a summary of the dataframe's structure
    column_summary = "Column Names and Data Types:\n"
    for col, dtype in column_info.items():
        column_summary += f"- {col}: {dtype}\n"

    # Unique values for categorical features
    unique_values_str = "\nUnique Values for Categorical Features:\n"
    for col, unique_values in unique_values.items():
        unique_values_str += f"- {col}: {uniques}\n"

    # Descriptive statistics for numerical features
    numerical_stats_str = "\nDescriptive Statistics for Numerical Features:\n"
    for col, stats in numerical_stats.items():
        numerical_stats_str += f"- {col}:\n"
        for stat_name, value in numerical_stats[col].items():
            numerical_stats_str += f"  {stat_name}: {value}\n"
    # Define the system prompt
    system_prompt = """
    You are an intelligent assistant that creates descriptive templates for transforming dataframe rows into coherent paragraphs.
    Analyze the provided dataframe structure and generate a template sentence that includes placeholders for each column.
    Ensure the template is contextually relevant and maintains grammatical correctness.
    """
    # Define the user prompt
    user_prompt = f"""
   Analyze the following dataframe structure and create a descriptive template with placeholders for each column.

    <column_summary>
    {column_summary}
    </column_summary>

    <unique_values>
    {unique_values_str}
    </unique_values>

    <numerical_stats>
    {numerical_stats_str}
    </numerical_stats>

    Use the exact column names from the column_summary in generating the variable names in the template,
    as they will be populated with the actual values in the dataset.

    Example Template about a Spotify dataset:
    "{{artist}} gained {{streams}} streams in the song '{{song}}' that was a hit in {{date}}."


    Output only the template without any explanation or introduction.
    The template's variables will be dynamically replaced so make sure they're formatted properly.
    """


    # Generate the template (with retries)
    retries = 3
    for attempt in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                temperature=0.3,
                max_tokens=1024,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            )
            template = response['choices'][0]['message']['content'].strip()
            return template
        except Exception as e:
            print(f"Error generating template (Attempt {attempt + 1}/{retries}): {e}")
            time.sleep(2)  # Wait before retrying

    return None


In [57]:
# Generate the template
template = generate_template(df, column_info, categorical_features, numerical_features, numerical_stats, unique_values)
print(template)


"The parcel with ID '{Parcel ID}' was sent by '{Sender Name}' from the address '{Sender Address}' to the recipient '{Recipient Name}' at '{Recipient Address}'. It weighs '{Weight}' and has dimensions of '{Dimensions}'. The shipping method used was '{Shipping Method}' and the parcel type is '{Parcel Type}'. Insurance was '{Insurance}' and a signature was '{Signature Required}'. The current delivery status is '{Delivery Status}' with an expected delivery date of '{Expected Delivery Date}'. The tracking number for this shipment is '{Tracking Number}', which was shipped on '{Shipment Date}' and is expected to be delivered by '{Delivery Date}'. The delivery fees amount to {Delivery Fees} and the payment status is '{Payment Status}'. The courier handling this shipment is '{Courier Name}'."


## Populating the Template with Actual Values

In [58]:
def populate_template(template, row):

    # Convert row to dictionary and replace NaN with 'N/A'
    row_dict = row.to_dict()
    for key, value in row_dict.items():
        if pd.isna(value):
            row_dict[key] = 'N/A'

    # Generate the populated template per row
    paragraph = template.format(**row_dict)
    return paragraph


In [59]:
# Apply the populate_template function to each row in the dataframe
df['paragraph'] = df.apply(lambda row: populate_template(template, row), axis=1)


In [60]:
df['paragraph'][0]

'"The parcel with ID \'P605730\' was sent by \'John Doe\' from the address \'123 Sender St, City 5\' to the recipient \'Jane Smith\' at \'456 Recipient Rd, City 10\'. It weighs \'5kg - 10kg\' and has dimensions of \'Small (30cm x 30cm)\'. The shipping method used was \'Express\' and the parcel type is \'Document\'. Insurance was \'Yes\' and a signature was \'Yes\'. The current delivery status is \'Out for Delivery\' with an expected delivery date of \'2024-11-15\'. The tracking number for this shipment is \'TRK10001\', which was shipped on \'2024-11-12\' and is expected to be delivered by \'2024-11-14\'. The delivery fees amount to 15.0 and the payment status is \'Paid\'. The courier handling this shipment is \'Mike Johnson\'."'

## Saving the Populated Templates (Natural Language Texts)

In [65]:
def save_content_to_txt(paragraphs, txt_filename):
    try:
        with open(txt_filename, 'w', encoding='utf-8') as file:

            for para in paragraphs:
                file.write(para + '\n\n')  # Double newline for separation

        print(f"All content has been saved as '{txt_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to text file: {e}")


In [66]:
def save_content_to_pdf(paragraphs, pdf_filename):
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.set_font("Arial", size=12)

        # Add each paragraph
        for para in paragraphs:
            pdf.multi_cell(0, 10, para)
            pdf.ln()  # Add a line break between paragraphs

        pdf.output(pdf_filename)
        print(f"All content has been saved as '{pdf_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to PDF: {e}")

In [67]:
# Save the paragraphs to both text and PDF files
save_content_to_txt(df['paragraph'].to_list(),txt_filename='nlg.txt')

save_content_to_pdf(df['paragraph'].to_list(), pdf_filename='nlg.pdf')

All content has been saved as 'nlg.txt'.

All content has been saved as 'nlg.pdf'.

